In [5]:
import pandas as pd
import numpy as np

In [52]:
lag = 4
trash_key = tuple([0]*(len(new_cols)-2-lag))
trash_key

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [6]:
train = pd.read_csv('./train.csv')

In [96]:
def _get_leak(df, new_cols, cols, lag=0):
    # trash_key = tuple([0]*(len(new_cols)-2-lag))
    d1 = df[new_cols[:-lag-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[new_cols[lag+2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    # d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    # d3 = d3[d3['key'] != trash_key]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [111]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]


new_cols = ['c928b4b74','8e4d0fe45', '6c0e0801a', '02861e414',
       'aac52d8d9', '041c5d0c9', 'd7875bb6c', 'e7c0cfd0f', 'd48c08bda',
       '0c9462c08', '57dd44c29', 'a93118262', '850027e38', 'db3839ab0',
       '27461b158', '32174174c', '9306da53f', '95742c2bf', '5831f4c76',
       '1e6306c7c', '06393096a', '13bdd610a', 'd7d314edc', '9a07d7b1f',
       '4d2671746', '822e49b95', '3c8a3ced0', '83635fb67', '1857fbccf',
       'c4972742d', 'b6c0969a2', 'e78e3031b', '36a9a8479', 'e79e5f72c',
       '092271eb3', '74d7f2dc3', '277ef93fc', 'b30e932ba', '8f57141ec',
       '350473311']

'''
new_cols = ['9884166a7', 'beb7f98fd', 'fd99c18b5', 'd83a2b684', '18c35d2ea',
       '0c8063d63', '400e9303d', 'c976a87ad', '8a088af55', '5f341a818',
       '5dca793da', 'db147ffca','762cbd0ab', 'fb5a3097e', '8c0a1fa32',
       '47cd6e6e4', '01005e5de', '50e4f96cf', 'f58fb412c', 'a1db86e3b',
       'f514fdb2e', '7a7da3079', 'bb1113dbb']
'''

# pattern_116 = pd.read_csv('./new_leak_patterns/pattern_1166666.66.csv')
# new_cols = list(pattern_116.columns)[1:-1]
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train_and_test = train
# train = train.drop(['target'],axis=1)
# train_and_test = pd.concat([train,test],axis=0)
# train_and_test = train_and_test.reset_index(drop=True)
total_leak = train_and_test[['ID']].copy()
max_lag = 38
for i in np.arange(max_lag):
    c = "leaked_target_"+str(i)
    total_leak[c] = _get_leak(train_and_test,cols,cols,i)
max_lag = 38
for i in np.arange(max_lag):
    c = "leaked_target_"+str(i+38)
    total_leak[c] = _get_leak(train_and_test,new_cols,cols,i)

In [112]:
new_cols

['c928b4b74',
 '8e4d0fe45',
 '6c0e0801a',
 '02861e414',
 'aac52d8d9',
 '041c5d0c9',
 'd7875bb6c',
 'e7c0cfd0f',
 'd48c08bda',
 '0c9462c08',
 '57dd44c29',
 'a93118262',
 '850027e38',
 'db3839ab0',
 '27461b158',
 '32174174c',
 '9306da53f',
 '95742c2bf',
 '5831f4c76',
 '1e6306c7c',
 '06393096a',
 '13bdd610a',
 'd7d314edc',
 '9a07d7b1f',
 '4d2671746',
 '822e49b95',
 '3c8a3ced0',
 '83635fb67',
 '1857fbccf',
 'c4972742d',
 'b6c0969a2',
 'e78e3031b',
 '36a9a8479',
 'e79e5f72c',
 '092271eb3',
 '74d7f2dc3',
 '277ef93fc',
 'b30e932ba',
 '8f57141ec',
 '350473311']

In [113]:
total_leak = total_leak.drop(['ID'],axis=1)
total_leak = total_leak.replace(0,np.nan)

In [114]:
total_leak['compiled_leak'] = np.nan
for i in np.arange(38):
    c = total_leak['compiled_leak'].isna()
    leak_target = 'leaked_target_'+str(i)
    total_leak.loc[c,'compiled_leak'] = total_leak.loc[c,leak_target]

total_leak['compiled_leak_new'] = np.nan
for i in np.arange(38):
    c = total_leak['compiled_leak_new'].isna()
    leak_target = 'leaked_target_'+str(i+38)
    total_leak.loc[c,'compiled_leak_new'] = total_leak.loc[c,leak_target]
# sum(train_leak['compiled_leak'] == train['target'])/sum(train_leak['compiled_leak'].notna())

In [115]:
pd.set_option('display.max_columns',100)
total_leak['target'] = train['target']
total_leak.loc[total_leak['compiled_leak'].isna() & total_leak['compiled_leak_new'].notna()]

,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,leaked_target_9,leaked_target_10,leaked_target_11,leaked_target_12,leaked_target_13,leaked_target_14,leaked_target_15,leaked_target_16,leaked_target_17,leaked_target_18,leaked_target_19,leaked_target_20,leaked_target_21,leaked_target_22,leaked_target_23,leaked_target_24,leaked_target_25,leaked_target_26,leaked_target_27,leaked_target_28,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,leaked_target_38,leaked_target_39,leaked_target_40,leaked_target_41,leaked_target_42,leaked_target_43,leaked_target_44,leaked_target_45,leaked_target_46,leaked_target_47,leaked_target_48,leaked_target_49,leaked_target_50,leaked_target_51,leaked_target_52,leaked_target_53,leaked_target_54,leaked_target_55,leaked_target_56,leaked_target_57,leaked_target_58,leaked_target_59,leaked_target_60,leaked_target_61,leaked_target_62,leaked_target_63,leaked_target_64,leaked_target_65,leaked_target_66,leaked_target_67,leaked_target_68,leaked_target_69,leaked_target_70,leaked_target_71,leaked_target_72,leaked_target_73,leaked_target_74,leaked_target_75,compiled_leak,compiled_leak_new,target
38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500000.0,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500000.00,18000000.00
163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5800000.0,NaN,NaN,NaN,NaN,5800000.00,60000.00
182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217666.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217666.66,1000000.00
185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000000.0,6000000.0,NaN,NaN,NaN,NaN,NaN,6000000.0,NaN,NaN,6000000.0,NaN,NaN,6000000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000000.00,6000000.00
187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.00,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.00,11600000.00
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,700000.0,700000.0,700000.0,700000.00,NaN,NaN,700000.00,27200000.00
536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000000.00,8000000.00
605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1013333.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10133

In [95]:
# useful_leak_columns = ['64e483341','9df4daa99','8c5025c23','8e4d0fe45','b850c3e18','18562fc62','f190486d6']
new_column = []
indexs = []
for column in list(train.columns)[2:]:
    if sum(train[column] == 1756000.0) > 0:
        new_column.append(column)
        indexs.extend(list(train.index[train[column] == 1756000.0]))

In [19]:
train.iloc[[586,2893,1195,4434,2143,1678,2970,4328,4368,1961,1753]][['83c3779bf', 'aa164b93b', 'f1e0ada11', '8f3740670',
       '342e7eb03', '2b2a10857', 'f39074b55', '1184df5c2', 'fec5eaf1a',
       'a8b721722', 'aca228668']]

,83c3779bf,aa164b93b,f1e0ada11,8f3740670,342e7eb03,2b2a10857,f39074b55,1184df5c2,fec5eaf1a,a8b721722,aca228668
586,1756000.0,390000.0,1000000.0,390000.0,390000.0,390000.0,390000.0,390000.0,390000.0,390000.0,390000.0
2893,0.0,1756000.0,0.0,0.0,80000.0,390000.0,390000.0,390000.0,390000.0,390000.0,390000.0
1195,0.0,1000000.0,1756000.0,390000.0,0.0,390000.0,390000.0,390000.0,390000.0,390000.0,390000.0
4434,0.0,298000.0,3556000.0,1756000.0,0.0,80000.0,390000.0,390000.0,390000.0,390000.0,390000.0
2143,3556000.0,1540000.0,298000.0,1000000.0,1756000.0,0.0,390000.0,390000.0,390000.0,390000.0,390000.0
1678,0.0,0.0,2368000.0,1540000.0,298000.0,1756000.0,390000.0,390000.0,390000.0,390000.0,390000.0
2970,0.0,2600000.0,1218500.0,0.0,0.0,754000.0,1756000.0,0.0,0.0,80000.0,390000.0
4328,1218500.0,0.0,2600000.0,1000000.0,0.0,0.0,1000000.0,1756000.0,2368000.0,0.0,390000.0
4368,754000.0,0.0,0.0,2368000.0,0.0,3556000.0,390000.0,390000.0,1756000.0,390000.0,390000.0
1961,2812000.0,1600000.0,0.0,0.0,2600000.0,0.0,1540000.0,298000.0,0.0,1756000.0,390000.0


In [126]:
pd.set_option('display.max_columns',30)
train.iloc[indexs][new_column]

,f7faf2d9f,342e7eb03,7e1c4f651,8f3740670,1184df5c2,aca228668,2b2a10857,fc99f9426,83c3779bf,aa164b93b,4edc3388d,8895ea516,bf59c51c3,0ff32eb98,a8b721722,fa9d6b9e5,f41f0eb2f,f1e0ada11,b88e5de84,26e998afd,fec5eaf1a,82775fc92,a8b590c6e,f39074b55,9e2040e5b,f7f553aea
2143,1756000.0,1756000.0,0.0,1000000.0,390000.0,390000.0,0.0,0.0,3556000.0,1540000.0,0.0,0.0,0.0,0.0,390000.0,0.0,0.0,298000.0,0.0,0.0,390000.0,0.0,0.0,390000.0,0.0,1500000.0
2143,1756000.0,1756000.0,0.0,1000000.0,390000.0,390000.0,0.0,0.0,3556000.0,1540000.0,0.0,0.0,0.0,0.0,390000.0,0.0,0.0,298000.0,0.0,0.0,390000.0,0.0,0.0,390000.0,0.0,1500000.0
1678,0.0,298000.0,1756000.0,1540000.0,390000.0,390000.0,1756000.0,0.0,0.0,0.0,2368000.0,0.0,0.0,0.0,390000.0,0.0,0.0,2368000.0,0.0,0.0,390000.0,0.0,0.0,390000.0,0.0,0.0
4434,0.0,0.0,0.0,1756000.0,390000.0,390000.0,80000.0,0.0,0.0,298000.0,0.0,0.0,0.0,0.0,390000.0,0.0,0.0,3556000.0,0.0,0.0,390000.0,0.0,0.0,390000.0,0.0,1756000.0
4328,0.0,0.0,0.0,1000000.0,1756000.0,390000.0,0.0,0.0,1218500.0,0.0,0.0,0.0,0.0,0.0,0.0,1500000.0,2368000.0,2600000.0,0.0,0.0,2368000.0,1756000.0,0.0,1000000.0,0.0,0.0
1753,0.0,0.0,0.0,0.0,1600000.0,1756000.0,0.0,0.0,0.0,0.0,0.0,1756000.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1678,0.0,298000.0,1756000.0,1540000.0,390000.0,390000.0,1756000.0,0.0,0.0,0.0,2368000.0,0.0,0.0,0.0,390000.0,0.0,0.0,2368000.0,0.0,0.0,390000.0,0.0,0.0,390000.0,0.0,0.0
1828,1240000.0,1240000.0,0.0,0.0,0.0,0.0,0.0,1756000.0,0.0,0.0,0.0,0.0,1756000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586,0.0,390000.0,0.0,390000.0,390000.0,390000.0,390000.0,0.0,1756000.0,390000.0,1500000.0,0.0,0.0,0.0,390000.0,0.0,0.0,1000000.0,0.0,0.0,390000.0,0.0,0.0,390000.0,1756000.0,0.0
2893,0.0,80000.0,0.0,0.0,390000.0,390000.0,390000.0,0.0,0.0,1756000.0,0.0,0.0,0.0,0.0,390000.0,0.0,0.0,0.0,0.0,1756000.0,390000.0,0.0,0.0,390000.0,0.0,0.0


In [11]:
# train = pd.read_csv('./train.csv')
pd.set_option('display.max_columns',200)
train.loc[train['target'] == 3556000.0,new_column]

,f7faf2d9f,342e7eb03,7e1c4f651,8f3740670,1184df5c2,aca228668,2b2a10857,fc99f9426,83c3779bf,aa164b93b,4edc3388d,8895ea516,bf59c51c3,0ff32eb98,a8b721722,fa9d6b9e5,f41f0eb2f,f1e0ada11,b88e5de84,26e998afd,fec5eaf1a,82775fc92,a8b590c6e,f39074b55,9e2040e5b,f7f553aea


In [49]:
train = pd.read_csv('./train.csv')
total_leak['target'] = train['target']
total_leak.loc[total_leak['compiled_leak'].isna() & total_leak['compiled_leak_new'].notna()]


,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,leaked_target_9,...,leaked_target_69,leaked_target_70,leaked_target_71,leaked_target_72,leaked_target_73,leaked_target_74,leaked_target_75,compiled_leak,compiled_leak_new,target
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30500000.00,8000000.0
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600000.00,3000000.0
187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.00,11600000.0
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1000000.0,NaN,1000000.00,NaN,NaN,1000000.00,27200000.0
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000000.00,10000000.0
662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000000.00,1000000.0
670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4000000.00,5000000.0
692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000000.00,100000.0
984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,618000.00,160000.0
1742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1490000.00,560000.0


In [18]:
test_leak_new = total_leak.iloc[4459:]
test_leak_new = test_leak_new.reset_index(drop=True)
test_leak_new.to_csv('./test_leak_new.csv',index=False)

In [19]:
train_leak_new = total_leak.iloc[:4459]
train_leak_new.to_csv('./train_leak_new.csv',index=False)

In [20]:
sum(test_leak_new['compiled_leak'].notna())

11824

In [21]:
sum(train_leak_new['compiled_leak'].notna())

3820